<a href="https://colab.research.google.com/github/MosaicGroupCMU/African-MRS-Tutorials/blob/main/Google-Colab/Answers/6_QE_Vibrational_Modes_H2O_Answers_Full_Install.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Quantum ESPRESSO and ASE

**Run the cell below at the start of today's class to install QE and ASE.**

Quantum ESPRESSO is a plane wave code, which uses Fourier transforms to solve equations in plane wave space. Here we install libraries for fast Fourier transforms (FFTs).

The Atomic Simulation Environment (ASE) is a set of tools for running, visualizing, and analyzing simulations.


In [1]:
# eliminate text output during installation
%%capture

# install mathematical libraries to peform fast Fourier transforms
!apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc

In [2]:
%%capture

# navigate to main directory named '/content/'
%cd /content/

# download the pre-compiled files in compressed format (under Linux)
#! wget 'https://docs.google.com/uc?export=download&id=1kw_CJMjP6ggDZXDNp5phAqCPpoe2WXCA' -O qe-lite.tgz
!gdown 'https://drive.google.com/uc?export=download&id=13l-Kiyg-F6aYb5lF8M3RsE1hSnLRdGna' -O qe-lite.tgz

# unpack the compressed files (under Linux)
! tar -xvzf qe-lite.tgz

# clean up some files
! rm -rf sample_data qe-lite.tgz

In [3]:
%%capture

# install the Atomic simulation environment - may take ~3 mins
# ! apt install ase
! pip install git+https://gitlab.com/ase/ase

---
# Vibrational Modes of Molecules and Extended Systems

Prerequisite: `5_QE_Vibrational_Modes_H2_Answers_Full_Install.ipynb`

Contributors: [Lorenzo Bastonero](https://github.com/bastonero), [Jessica Wen](https://github.com/JessicaWen-PhD)

This easy workbook is designed for people who are unfamiliar with Python and terminal commands.

If this notebook is too easy for you, there is a [medium-level workbook]() designed for people who have done Python and worked with terminal commands before, but are unfamiliar with Quantum ESPRESSO; if you have already dabbled in some Quantum ESPRESSO, you can try the [hard version]() of the workbook.

## Learning Objectives

In this tutorial, you will lean how to calculate infrared and Raman spectra from first-principles in the non-resonant regime and harmonic approximation by using the Quantum ESPRESSO distribution.

Vibrational spectra, such as infrared (IR) and Raman, provide access to the collective atomic vibrations of materials. Each peak corresponds to a particular collective vibration, referred to as _normal mode_. These normal modes are characteristics of the each specific system, and can be used to characterize real samples (e.g., for quality control).

So, the question now is: how can we compute the _frequency_ and _intensity_ of the vibrations?

We have already found the method to compute the frequency of vibrations from `5_QE_Vibrational_Modes_H2_Answers_Full_Install.ipynb`.

Now we can calculate the _intensity_ of the vibrations using water as an example.

# The harmonic approximation

Remember, atoms in molecules and solids vibrate around their _equilibrium_ positions.

![n2harmonicoscillation.png](https://upload.wikimedia.org/wikipedia/commons/a/aa/N2ground.png)

# Setup

This part installs libraries for numerical calculations and plotting.

In [4]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

This part uploads the pseudopotentials needed for our calculations.

## $H_2O$: infrared spectra

### Step 1: define a structure

Let's define the structure of water using ASE.

In [5]:
from ase import build
from ase.build import molecule
from ase.visualize import view
from ase.io import read, write

h2o = molecule('H2O', vacuum=3.0, pbc=True)

view(h2o, viewer='x3d')

In [7]:
# h2o.get_atomic_numbers()
h2o.get_positions()

array([[3.      , 3.763239, 3.596309],
       [3.      , 4.526478, 3.      ],
       [3.      , 3.      , 3.      ]])

### Step 2: optimize the atomic positions

Start with the SCF relax calculation.

In [19]:
# create calculation folder and navigate into it
%mkdir -p /content/water/
%cd /content/water/

# create input and write it into the file h2.scf.in
qe_input = """
&control
  prefix='h2o',
  calculation='relax',
  verbosity='high',
  pseudo_dir = '/content/qe-lite/pseudopotentials/',
  outdir='/content/water/'
  forc_conv_thr=1.0e-3,
  tstress=true, ! to extract the stresses so we know how to change the cell parameters
  tprnfor=true, ! to extract the forces
/
&system
  occupations='fixed',
  ecutwfc = 30.0,
  ecutrho = 240.0,
  ibrav = 1,
  celldm(1) = 10,
  nat = 3,
  ntyp = 2,
/
&electrons
  conv_thr = 1e-12,
/
&ions
  ion_dynamics = 'bfgs',
/
ATOMIC_SPECIES
 H  1.00784  H.upf
 O  15.999   O.upf
ATOMIC_POSITIONS (angstrom)
 O 3.000000 3.763239 3.596309
 H 3.000000 4.526478 3.000000
 H 3.000000 3.000000 3.000000
K_POINTS (automatic)
  1 1 1 0 0 0
"""

with open("h2o.scf.in", "w") as f:
    f.write(qe_input)

# print the content of the input file (under Linux)
! cat h2o.scf.in

/content/water

&control
  prefix='h2o',
  calculation='relax',
  verbosity='high',
  pseudo_dir = '/content/qe-lite/pseudopotentials/',
  outdir='/content/water/'
  forc_conv_thr=1.0e-3,
  tstress=true, ! to extract the stresses so we know how to change the cell parameters
  tprnfor=true, ! to extract the forces
/
&system
  occupations='fixed',
  ecutwfc = 30.0,
  ecutrho = 240.0,
  ibrav = 1,
  celldm(1) = 10,
  nat = 3,
  ntyp = 2,
/
&electrons
  conv_thr = 1e-12,
/
&ions
  ion_dynamics = 'bfgs',
/
ATOMIC_SPECIES
 H  1.00784  H.upf
 O  15.999   O.upf
ATOMIC_POSITIONS (angstrom)
 O 3.000000 3.763239 3.596309
 H 3.000000 4.526478 3.000000
 H 3.000000 3.000000 3.000000
K_POINTS (automatic)
  1 1 1 0 0 0


In [20]:
! /content/qe-lite/bin/pw.x -in h2o.scf.in > h2o.scf.out # run the relax calculation
! cat h2o.scf.out


     Program PWSCF v.7.4 starts on 18Dec2024 at  9:17:34 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Serial version
     7660 MiB available memory on the printing compute node when the environment starts

     Reading input from h2o.scf.in

     Current dimensions of program PWSCF are:
     Max number of different atomic species (ntypx) = 10
     Max number of k-points (npk) =  40000
     Max angular momentum in pseudopotentials (lmaxx) =  4
     file O.upf: wavefunction(s)  2P renormalized


     G-vector sticks info
     -

### Step 3: calculate vibrational modes and effective charges

In [21]:
ph_inputs = """
&INPUTPH
  tr2_ph=1.0e-15,
  prefix='h2o',
  verbosity='high',
  epsil=.true.
  alpha_mix(1)=0.4
  ldisp=.true
  nq1=1
  nq2=1
  nq3=1
/
"""

with open('h2o.ph.in', 'w+') as handle:
    handle.write(ph_inputs)

! /content/qe-lite/bin/ph.x -in h2o.ph.in > h2o.ph.out # run the ph.x executable

! cat h2o.ph.out


     Program PHONON v.7.4 starts on 18Dec2024 at  9:18: 5 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Serial version
     7610 MiB available memory on the printing compute node when the environment starts

     Reading input from h2o.ph.in

     Reading xml data from directory:

     ./h2o.save/

     file O.upf: wavefunction(s)  2P renormalized

     IMPORTANT: XC functional enforced from input :
     Exchange-correlation= PBE
                           (   1   4   3   4   0   0   0)
     Any further DFT definition will be 

Analyze the results.

In [22]:
from ase.io import espresso

with open('h2o.ph.out', 'r') as handle:
    ph_h2o_results = espresso.read_espresso_ph(handle)

print("The vibronic frequencies are:")

thz_to_invcm = 33.3564095198152 # convert THz to inverse cm
ph_h2o_results[1]['freqs'] * thz_to_invcm

The vibronic frequencies are:


array([-251.21666003, -223.40381892, -184.94604691,  -15.36969953,
         15.45349083,   19.15648592, 1583.54934333, 3617.73467296,
       3718.86757071])

In [23]:
print("The effective charges are:")
ph_h2o_results[1]['borneffcharge']

The effective charges are:


[array([[-5.9924e-01, -6.0000e-05, -4.0000e-05],
        [ 1.0000e-05, -3.3198e-01, -7.0000e-05],
        [-7.0000e-05, -1.0000e-04, -2.1286e-01]]),
 array([[ 2.9961e-01,  2.0000e-05,  2.0000e-05],
        [-1.0000e-05,  1.6600e-01,  6.3130e-02],
        [ 5.0000e-05,  1.1228e-01,  1.0649e-01]]),
 array([[ 2.9963e-01,  3.0000e-05,  2.0000e-05],
        [ 0.0000e+00,  1.6598e-01, -6.3060e-02],
        [ 2.0000e-05, -1.1218e-01,  1.0637e-01]])]

In [24]:
ph_h2o_results[1]['borneffcharge'][0].trace(), ph_h2o_results[1]['borneffcharge'][1].trace()

(-1.14408, 0.5720999999999999)

### Step 4: calculate vibrational intensities

In [25]:
dynmat_inputs = """
&INPUT
  fildyn='matdyn1',
  asr='zero-dim',
  filout='h2o.dynmat.out'
/
"""

with open('h2o.dynmat.in', 'w+') as handle:
    handle.write(dynmat_inputs)

! /content/qe-lite/bin/dynmat.x -in h2o.dynmat.in > h2o.dynmat.out # run the dynmat.x executable

! cat h2o.dynmat.out


     Program DYNMAT v.7.4 starts on 18Dec2024 at  9:22: 1 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Serial version
     7580 MiB available memory on the printing compute node when the environment starts


     Reading Dynamical Matrix from file matdyn1
     ...Force constants read
     ...epsilon and Z* read
     Acoustic Sum Rule: || Z*(ASR) - Z*(orig)|| =    7.832723E-01
     Acoustic Sum Rule: ||dyn(ASR) - dyn(orig)||=    1.212016E-02
     A direction for q was not specified:TO-LO splitting will be absent

     Polariza

The result should be comparable to the following:

| mode | [cm-1]  |  [THz]    |   IR    |
|------|---------|-----------|---------|
|    1 |  -0.00  |  -0.0000  | 0.0000  |
|    2 |  -0.00  |  -0.0000  | 0.0000  |
|    3 |  -0.00  |  -0.0000  | 0.0000  |
|    4 |   0.00  |   0.0000  | 0.0000  |
|    5 |   0.00  |   0.0000  | 0.0000  |
|    6 |   0.00  |   0.0000  | 0.0000  |
|    7 | 1676.57 |  50.2622  | 1.1143  |
|    8 | 3724.30 | 111.6518  | 0.0041  |
|    9 | 3841.84 | 115.1754  | 2.2215  |

![vibrationalspectra.png](https://arxiv.org/html/2312.15211v2/x4.png)

Taken from _Kovács, D. V. et al, (2024) arXiv:2312.15211_